In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')
import RamanGraphene as rg


In [2]:
x_axis_300_g = np.loadtxt('calibrated_axis_300gr.dat')

lambda_laser = 632.8
x_axis_300_g_raman = rg.convert_nm_to_raman(x_axis_300_g, lambda_laser)

reference_flake_1 = np.loadtxt('Raman_suspended_graphene_sample_6_633nm_1mW_300gr_mm_RT_300s_flake_1_1.dat')[:,1]
map_flake_1 = rg.get_map_data('../09_12_2024/Flake_1/LRVI_H12P1_1-4__2024-12-09_17-25-27_tot.dat')

reference_flake_2 = np.loadtxt('Raman_suspended_graphene_sample_6_633nm_1mW_300gr_mm_RT_300s_flake_2_1.dat')[:,1]
map_flake_2 = rg.get_map_data('../09_12_2024/Flake_2/LRVI_H12P1_1-4_map2_2024-12-09_19-51-12_tot.dat')


In [3]:
%matplotlib qt

# Visualize raw data
plt.plot(x_axis_300_g_raman, reference_flake_1)
plt.xlabel('Raman shift (cm^-1)')
plt.ylabel('Intensity (a.u.)')
plt.title('Raman spectrum of suspended graphene sample 6, flake 1')

plt.show()

In [4]:
#This cell is to make a file that can be used with the matlab script 

ones_column = np.ones_like(x_axis_300_g_raman)
data_to_save = np.column_stack((x_axis_300_g_raman, ones_column, reference_flake_2))

# Save the data to a text file
np.savetxt('Reference_sample_6_flake_2.txt', data_to_save, fmt='%.6e', delimiter='\t')

In [4]:
%matplotlib qt
#Fit for the reference spectrum of sample 6, flake 1

omega_G = 1581
omega_2D = 2630

x_min = 1400
x_max = 1850


x_ROI = x_axis_300_g_raman[(x_axis_300_g_raman > x_min) & (x_axis_300_g_raman < x_max)]
y_ROI = reference_flake_1[(x_axis_300_g_raman > x_min) & (x_axis_300_g_raman < x_max)]

results_G, results_2D, params1 = rg.raman_Gr_fit(x_ROI, y_ROI, omega_G, omega_2D)
#rg.plot_raman_fit(results_G, results_2D, params1, x_ROI, y_ROI, 'Raman spectrum of suspended graphene sample 6, flake 1', omega_G, omega_2D)

RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 1400.

In [4]:
rg.print_fit_results(params1, 3)

G peak: 
 off_G = -108.378 ± 11.800 
 ω_G = 1582.168 ± 0.053 cm⁻¹ 
 Γ_G = 14.749 ± 0.174 cm⁻¹ 
 I_G = 148145.545 ± 1435.341
2D peak: 
 off_2D = -30.308 ± 16.960 
 ω_2D- = 2632.598 ± 0.060 cm⁻¹, ω_2D+ = 2645.616 ± 0.060 cm⁻¹ 
 Γ_2D = 20.940 ± 0.104 cm⁻¹ 
 I_2D = 1135942.034 ± 2708.461 
 I_2D-/I_2D+ = 2.381 ± 0.005


In [5]:
#Draw the Raman map of sample 6, flake 1

omega_G = 1581
omega_2D = 2630

x_min = 1500
x_max = 2800


noise, signal = rg.separate_data_noise(map_flake_2)
data_map = signal 
map_size = (15,15)
_,_, the_map = rg.make_map(x_axis_300_g_raman, data_map, map_size, x_min, x_max)

data_map = data_map / np.max(the_map)

plt.rcParams.update({'font.size': 16})
rg.draw_map(x_axis_300_g_raman, data_map, map_size, x_min, x_max, '',omega_G, omega_2D)
plt.tight_layout()

Clicked at coordinates: (7.66, 5.39)
Clicked at indices: (8, 5)


In [17]:
x_min_G = 1520
x_max_G = 1650

x_min_2D = 2550
x_max_2D = 2800

_,_,the_map_G = rg.make_map(x_axis_300_g_raman, data_map, map_size, x_min_G, x_max_G)
_,_,the_map_2D = rg.make_map(x_axis_300_g_raman, data_map, map_size, x_min_2D, x_max_2D)

relative_map = the_map_2D / the_map_G
relative_map[4,8] = np.mean((relative_map[5,8], relative_map[4,7])) #There is a huge spike in that spot
relative_map = relative_map / np.max(relative_map)
plt.figure()
plt.imshow(relative_map, interpolation='nearest', cmap=plt.cm.jet)
plt.xlabel(f'$\mu$m')
plt.ylabel(f'$\mu$m')
plt.colorbar(label = 'Integrated intensity (a.u.)')
plt.tight_layout()
plt.show()

<>:15: SyntaxWarning: invalid escape sequence '\m'
<>:16: SyntaxWarning: invalid escape sequence '\m'
<>:15: SyntaxWarning: invalid escape sequence '\m'
<>:16: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_43395/1888340160.py:15: SyntaxWarning: invalid escape sequence '\m'
  plt.xlabel(f'$\mu$m')
/tmp/ipykernel_43395/1888340160.py:16: SyntaxWarning: invalid escape sequence '\m'
  plt.ylabel(f'$\mu$m')


15.880215515832893


In [23]:
#Fit for the reference spectra of G peak

x_G_peak = np.loadtxt('calibrated_axis_900gr_center_at_700nm.dat')
x_G_peak = rg.convert_nm_to_raman(x_G_peak, lambda_laser)

y_G_peak_flake2 = np.loadtxt('Raman_suspended_graphene_sample_6_633nm_1mW_900gr_mm_RT_600s_flake_1_center_at_700nm_1.dat')[:,1]

In [24]:
#Plot raw data for G peak

plt.plot(x_G_peak, y_G_peak_flake2)
plt.xlabel('Raman shift (cm^-1)')
plt.ylabel('Intensity (a.u.)')
plt.title('Raman spectrum of suspended graphene sample 6, flake 2, G peak')
plt.show()

In [25]:
results_G_peak, garbage, params2 = rg.raman_Gr_fit(x_G_peak, y_G_peak_flake2, omega_G, 1900)
rg.plot_raman_fit(results_G_peak, garbage, params2, x_G_peak, y_G_peak_flake2, 'Raman spectrum of suspended graphene sample 6, flake 2, G peak', omega_G, 1900)

In [26]:
print(f'\n G peak offset: {params2[0][0]:.2f} \n G peak position: {params2[0][1]:.2f} cm^(-1)\n G peak width: {params2[0][2]:.2f} \n G peak area: {params2[0][3]:.2f} \n')


 G peak offset: -63.65 
 G peak position: 1582.14 cm^(-1)
 G peak width: 12.79 
 G peak area: 148373.65 



In [27]:
#Fit for the reference spectra of 2D peak

x_2D_peak = np.loadtxt('calibrated_axis_900gr_center_at_760nm.dat') #This calibration might be wrong
x_2D_peak = rg.convert_nm_to_raman(x_2D_peak, lambda_laser)

y_2D_peak_flake2 = np.loadtxt('Raman_suspended_graphene_sample_6_633nm_1mW_900gr_mm_RT_600s_flake_1_center_at_760nm_1.dat')[:,1]

In [28]:
plt.plot(x_2D_peak, y_2D_peak_flake2)
plt.xlabel('Raman shift (cm^-1)')
plt.ylabel('Intensity (a.u.)')
plt.title('Raman spectrum of suspended graphene sample 6, flake 2, 2D peak')
plt.show()

In [29]:
garbage, results_2D_peak, params3 = rg.raman_Gr_fit(x_2D_peak, y_2D_peak_flake2, 2250, omega_2D)
rg.plot_raman_fit(garbage, results_2D_peak, params3, x_2D_peak, y_2D_peak_flake2, 'Raman spectrum of suspended graphene sample 6, flake 2, 2D peak', 2250, omega_2D)

In [30]:
print(f'\n 2D peak offset: {params3[1][0]:.2f} \n 2D peak position: {params3[1][1]:.2f} cm^(-1)\n 2D peak width: {params3[1][2]:.2f} \n 2D peak area: {params3[1][3]:.2f} \n 2D peak relative area: {params3[1][4]:.2f} \n 2D peak shift: {params3[1][5]:.2f} \n')


 2D peak offset: 7.30 
 2D peak position: 2632.00 cm^(-1)
 2D peak width: 13.34 
 2D peak area: 850900.11 
 2D peak relative area: 0.71 
 2D peak shift: 13.05 

